
    1.скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention
        
    2. поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.
    
    3. сделать разбиение набора данных не тренировочную и тестовую выборки
    4. провести uplift-моделирование 3 способами:
        одна модель с признаком коммуникации (S-learner)
        модель с трансформацией таргета
        вариант с двумя независимыми моделями
        
    5.в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% 3 моделей
    *для модели S-learner построить зависимость таргета (конверсии - поле conversion) от значения uplift:
        сделать прогноз и получить uplift для тестовой выборки
        отсортировать тестовую выборку по uplift по убыванию
        разбить на децили (pandas qcut вам в помощь)
        для каждого дециля посчитать среднюю conversion



In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split

In [2]:
# !unzip data.csv.zip

In [3]:
df = pd.read_csv('data.csv')
df

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0
...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web,Discount,0
63996,5,38.91,0,1,Urban,1,Phone,Discount,0
63997,6,29.99,1,0,Urban,1,Phone,Discount,0
63998,1,552.94,1,0,Surburban,1,Multichannel,Buy One Get One,0


In [4]:
df.rename(columns = {'conversion' : 'target', 'offer' : 'treatment'}, inplace = True) 

In [5]:
df['treatment'].value_counts()

Buy One Get One    21387
Discount           21307
No Offer           21306
Name: treatment, dtype: int64

In [6]:
df['treatment'] = df['treatment'].map({'Buy One Get One':'1','Discount':'1', 'No Offer':'0'}).astype(int)

In [7]:
df['treatment'].value_counts()

1    42694
0    21306
Name: treatment, dtype: int64

In [8]:
df_clients = df.drop(['treatment','target'], axis=1)
df_clients.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel
0,10,142.44,1,0,Surburban,0,Phone
1,6,329.08,1,1,Rural,1,Web
2,7,180.65,0,1,Surburban,1,Web
3,9,675.83,1,0,Rural,1,Web
4,2,45.34,1,0,Urban,0,Web


### разделим данные на train/test

In [9]:
df_train, df_test= train_test_split(df, test_size=0.3,shuffle=True, random_state=0)

In [10]:
df_train.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
26161,1,571.68,0,1,Surburban,1,Phone,1,0
10495,12,61.40,1,0,Urban,0,Web,1,0
50047,3,465.79,0,1,Surburban,0,Web,1,0


In [11]:
df_train = df_train[['treatment','target']]
df_train.head()

,treatment,target
26161,1,0
10495,1,0
50047,1,0
56688,0,1
62760,1,0


In [12]:
df_test.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
11277,3,466.66,0,1,Urban,1,Phone,1,1
55819,8,44.25,0,1,Urban,0,Phone,1,0
43223,10,370.84,0,1,Urban,0,Phone,1,0
1351,3,114.27,1,0,Urban,1,Web,1,0
9247,5,230.93,1,1,Surburban,0,Phone,0,0


In [13]:
# Извлечение признаков
df_features = df_clients.copy()

indices_train = df_train.index
indices_test = df_test.index
indices_learn, indices_valid = train_test_split(indices_train, test_size=0.3, random_state=123)

In [14]:
df_features

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel
0,10,142.44,1,0,Surburban,0,Phone
1,6,329.08,1,1,Rural,1,Web
2,7,180.65,0,1,Surburban,1,Web
3,9,675.83,1,0,Rural,1,Web
4,2,45.34,1,0,Urban,0,Web
...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web
63996,5,38.91,0,1,Urban,1,Phone
63997,6,29.99,1,0,Urban,1,Phone
63998,1,552.94,1,0,Surburban,1,Multichannel


In [15]:
indices_train

Int64Index([26161, 10495, 50047, 56688, 62760, 58439, 55939,  6202,  8708,
            53518,
            ...
            57043, 41993, 32103, 30403, 21243, 45891, 52416, 42613, 43567,
             2732],
           dtype='int64', length=44800)

In [16]:
indices_test

Int64Index([11277, 55819, 43223,  1351,  9247, 46551,  8209, 27633, 51794,
            38249,
            ...
             5083, 14683, 60969, 51312, 16977,  1831, 37166,   169,  6977,
            10028],
           dtype='int64', length=19200)

In [17]:
df_train.groupby("treatment")['target'].describe()

,count,mean,std,min,25%,50%,75%,max
treatment,,,,,,,,
0,14982.0,0.106394,0.308352,0.0,0.0,0.0,0.0,1.0
1,29818.0,0.168724,0.374514,0.0,0.0,0.0,0.0,1.0


### Заведем переменные

In [18]:
X_train = df_features.loc[indices_learn, :]
y_train = df_train.loc[indices_learn, 'target']
treat_train = df_train.loc[indices_learn, 'treatment']

X_val = df_features.loc[indices_valid, :]
y_val = df_train.loc[indices_valid, 'target']
treat_val =  df_train.loc[indices_valid, 'treatment']


X_test = df_features.loc[indices_test, :]

cat_features = ['zip_code','channel']

models_results = {
    'approach': [],
    'uplift@10%': [],
    'uplift@20%': []
}

In [19]:
X_train.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel
27528,3,897.77,1,1,Surburban,1,Phone
30780,2,162.36,1,0,Surburban,0,Phone
40399,9,115.53,0,1,Rural,1,Web
32084,4,49.61,1,0,Surburban,0,Web
59420,3,214.00,0,1,Urban,1,Multichannel


Описание воздействия

In [20]:
treat_train.head()

27528    0
30780    0
40399    1
32084    1
59420    1
Name: treatment, dtype: int64

Описание целевого действия

In [21]:
y_train.head()

27528    0
30780    0
40399    0
32084    0
59420    0
Name: target, dtype: int64

In [22]:
ttt

NameError: name 'ttt' is not defined

## 1. Подходы с одной моделью

In [ ]:
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel

# sklift поддерживает любые модели, 
# которые удовлетворяют соглашениями scikit-learn
# Для примера воспользуемся catboost
from catboost import CatBoostClassifier


sm = SoloModel(
    CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features)
)

sm = sm.fit(X_train, y_train, treat_train)

uplift_sm = sm.predict(X_val)

sm_score = uplift_at_k(y_true=y_val, uplift=uplift_sm, treatment=treat_val, strategy='by_group', k=0.1)
sm_score_20 = uplift_at_k(y_true=y_val, uplift=uplift_sm, treatment=treat_val, strategy='by_group', k=0.2)

models_results['approach'].append('SoloModel')
models_results['uplift@10%'].append(sm_score)
models_results['uplift@20%'].append(sm_score_20)

# Получим условные вероятности выполнения целевого действия при взаимодействии для каждого объекта
sm_trmnt_preds = sm.trmnt_preds_
# И условные вероятности выполнения целевого действия без взаимодействия для каждого объекта
sm_ctrl_preds = sm.ctrl_preds_

# Отрисуем распределения вероятностей и их разность (uplift)
plot_uplift_preds(trmnt_preds=sm_trmnt_preds, ctrl_preds=sm_ctrl_preds);

Посмотрим на топ-признаки

In [ ]:
# С той же легкостью можно обратиться к обученной модели.
# Например, чтобы построить важность признаков:
sm_fi = pd.DataFrame({
    'feature_name': sm.estimator.feature_names_,
    'feature_score': sm.estimator.feature_importances_
}).sort_values('feature_score', ascending=False).reset_index(drop=True)

sm_fi

In [ ]:
sm_score

### Метрики

uplift@k

In [ ]:
n = 15
df = pd.DataFrame({
    'true': y_val[:n],
    'uplift': uplift_sm[:n],
    'treat': treat_val[:n]
})   
df 

strategy='overall'

In [ ]:
k = 0.7

uplift_at_k(y_true=df['true'], uplift=df['uplift'], treatment=df['treat'],
            # strategy='by_group',
            strategy='overall',
            k=k)

In [ ]:
df_k = df.sort_values(by='uplift', ascending=False).head(int(n * k))
df_k

In [ ]:
response_rate_treat = df_k[(df_k.treat == 1)].true.mean()
response_rate_control = df_k[(df_k.treat == 0)].true.mean()
response_rate_treat - response_rate_control

**strategy='by_group'**


In [ ]:
k = 4
uplift_at_k(y_true=df['true'], uplift=df['uplift'], treatment=df['treat'],
            strategy='by_group',
            # strategy='overall',
            k=k)

In [ ]:
df_k_treat = df[df.treat == 1]
df_k_treat = df_k_treat.sort_values(by='uplift', ascending=False).head(k)

df_k_control = df[df.treat == 0]
df_k_control = df_k_control.sort_values(by='uplift', ascending=False).head(k)

df_k_treat

In [ ]:
df_k_control

In [ ]:
response_rate_treat = df_k_treat.true.mean()
response_rate_control = df_k_control.true.mean()
response_rate_treat - response_rate_control

**Uplift curve**

In [ ]:
from sklift.viz import plot_uplift_curve


# with ideal curve
# perfect=True
plot_uplift_curve(
    y_val, uplift_sm, treat_val, 
#     perfect=True,
    perfect=False
);

##### Qini curve

In [ ]:
from sklift.viz import plot_qini_curve


# with ideal Qini curve (red line)
# perfect=True
plot_qini_curve(
    y_val, uplift_sm, treat_val,
    # perfect=True
    perfect=False
);

In [ ]:
from sklift.metrics import qini_auc_score


# коэффициент qini, или area under qini curve (AUQC)
auqc = qini_auc_score(y_val, uplift_sm, treat_val) 
print(f"Qini coefficient on full data: {auqc:.4f}")

##### Uplift by percentile

In [ ]:
from sklift.metrics import uplift_by_percentile

# uplift by decile, или  uplift by percentile, или uplift by bin.
uplift_by_percentile(y_val, uplift_sm,  treat_val, 
                     strategy='overall', 
                     total=True, std=True, bins=10)

In [ ]:
from sklift.viz import plot_uplift_by_percentile

# график uplift by percentile
plot_uplift_by_percentile(y_val, uplift_sm, treat_val, 
                          strategy='overall', kind='bar'); # line

In [ ]:
from sklift.metrics import weighted_average_uplift

# с помощью значений в столбцах uplift и n_treatment рассчитаем усредненный uplift на всей выборке, 
# взвешенный на размер целевой группы — weighted average uplift
uplift_full_data = weighted_average_uplift(y_val, uplift_sm, 
                                            treat_val, bins=10) 
print(f"weighted average uplift on full data: {uplift_full_data:.4f}")

### Трансформация классов

In [ ]:
from sklift.models import ClassTransformation


ct = ClassTransformation(
    CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features)
)
ct = ct.fit(X_train, y_train, treat_train)

uplift_ct = ct.predict(X_val)

ct_score = uplift_at_k(y_true=y_val, uplift=uplift_ct, treatment=treat_val, strategy='by_group', k=0.1)
ct_score_20 = uplift_at_k(y_true=y_val, uplift=uplift_ct, treatment=treat_val, strategy='by_group', k=0.2)

models_results['approach'].append('ClassTransformation')
models_results['uplift@10%'].append(ct_score)
models_results['uplift@20%'].append(ct_score_20)

In [ ]:
ct_score

In [ ]:
from sklift.viz import plot_uplift_curve


# with ideal curve
# perfect=True
plot_uplift_curve(
    y_val, uplift_ct, treat_val, 
    # perfect=True
    perfect=False
);

In [ ]:
# график uplift by percentile
plot_uplift_by_percentile(y_val, uplift_ct, treat_val, 
                          strategy='overall', kind='bar'); # line

### 2. Две независимые модели

In [ ]:
from sklift.models import TwoModels


tm = TwoModels(
    CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features),
    CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features),
    method='vanilla'  # независимые модели
)
tm = tm.fit(
    X_train, y_train, treat_train
)

uplift_tm = tm.predict(X_val)

tm_score = uplift_at_k(y_true=y_val, uplift=uplift_tm, treatment=treat_val, strategy='by_group', k=0.1)
tm_score_20 = uplift_at_k(y_true=y_val, uplift=uplift_tm, treatment=treat_val, strategy='by_group', k=0.2)

models_results['approach'].append('TwoModels')
models_results['uplift@10%'].append(tm_score)
models_results['uplift@20%'].append(tm_score_20)

plot_uplift_preds(trmnt_preds=tm.trmnt_preds_, ctrl_preds=tm.ctrl_preds_);

Посмотрим на результаты

In [ ]:
pd.DataFrame(data=models_results).sort_values('uplift@10%', ascending=False)

In [ ]:
import matplotlib.pyplot as plt

print('Solo Model')
plot_uplift_curve(y_val, uplift_sm, treat_val, perfect=False)
plt.show()
print('ClassTransform Model')
plot_uplift_curve(y_val, uplift_ct, treat_val, perfect=False)
plt.show()
print('Two Model')
plot_uplift_curve(y_val, uplift_tm, treat_val, perfect=False)
plt.show()